In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, LeakyReLU, UpSampling1D, Concatenate, Subtract
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Cropping1D
from tensorflow.keras.layers import Reshape
from tensorflow.nn import sigmoid
import os
import librosa
import numpy as np
import random
from concurrent.futures import ProcessPoolExecutor


In [25]:
model_path = "/Users/rei/Documents/Machine_Learning/MODELS/Unet/Shaking Through Sound Seperation/Shaking_Through_Model_01.keras"
trainDir = '/Users/rei/Documents/Machine_Learning/Data/Audio/Shaking_Through/Dataset/Train'
testDir = '/Users/rei/Documents/Machine_Learning/Data/Audio/Shaking_Through/Dataset/Test'


In [26]:
def crop(tensor, target_shape, match_feature_dim=True):
    shape = tf.shape(tensor)
    diff = shape - target_shape
    assert diff[1] >= 0 # Only positive difference allowed
    if diff[1] == 0:
        return tensor
    crop_start = diff // 2
    crop_end = diff - crop_start
    return tensor[:, crop_start[1]:-crop_end[1], :]

def AudioClip(x, training):
    if training:
        return x
    else:
        return tf.maximum(tf.minimum(x, 1.0), -1.0)

def difference_output(input_mix, featuremap, source_names, num_channels, filter_width, padding, activation, training):
    outputs = dict()
    sum_source = 0
    for name in source_names[:-1]:
        out = tf.keras.layers.Conv1D(num_channels, filter_width, activation=activation, padding=padding)(featuremap)
        outputs[name] = out
        sum_source += out

    last_source = crop(input_mix, sum_source.shape) - sum_source
    last_source = AudioClip(last_source, training)
    outputs[source_names[-1]] = last_source
    return outputs

In [27]:
def learned_interpolation_layer(input, padding, level):
    features = input.shape[2]
    weights = tf.Variable(tf.initializers.GlorotUniform()(shape=[features]), dtype=tf.float32, name="interp_" + str(level))
    weights_scaled = tf.nn.sigmoid(weights)
    counter_weights = 1.0 - weights_scaled

    conv_weights = tf.linalg.diag(weights_scaled)
    conv_weights = tf.expand_dims(conv_weights, axis=0)
    intermediate_vals = tf.linalg.matmul(input, conv_weights)
    
    counter_conv_weights = tf.linalg.diag(counter_weights)
    counter_conv_weights = tf.expand_dims(counter_conv_weights, axis=0)
    counter_intermediate_vals = tf.linalg.matmul(input, counter_conv_weights)

    output = tf.concat([intermediate_vals, counter_intermediate_vals], axis=1)
    
    if padding == "valid":
        output = output[:, :-1, :]

    return output



In [28]:
def create_encoder(input, num_layers, num_initial_filters, filter_size, input_filter_size, padding):
    enc_outputs = []
    current_layer = input
    current_layer = tf.keras.layers.Conv1D(num_initial_filters, input_filter_size, strides=1, activation=LeakyReLU(), padding=padding)(current_layer)
    enc_outputs.append(current_layer)

    for i in range(num_layers - 1):
        current_layer = tf.keras.layers.Conv1D(num_initial_filters + (num_initial_filters * i), filter_size, strides=1, activation=LeakyReLU(), padding=padding)(current_layer)
        current_layer = current_layer[:, ::2, :]  # Decimate by factor of 2
        enc_outputs.append(current_layer)

    return enc_outputs

In [29]:
def create_decoder(enc_outputs, num_layers, num_initial_filters, filter_size, merge_filter_size, padding, upsampling):
    current_layer = enc_outputs[-1]

    for i in range(num_layers - 1, 0, -1):
        if upsampling == 'linear':
            current_layer = tf.keras.layers.UpSampling1D(size=2)(current_layer)
        elif upsampling == 'learned':
            current_layer = learned_interpolation_layer(current_layer, padding=padding, level=i)

        current_layer = tf.concat([current_layer, enc_outputs[i - 1]], axis=2)
        current_layer = tf.keras.layers.Conv1D(num_initial_filters * (num_layers - i), merge_filter_size, strides=1, activation=LeakyReLU(), padding=padding)(current_layer)

    return current_layer



In [30]:
def get_output_layer(current_layer, output_type, source_names, num_channels, output_filter_size, padding, activation, training):
    if output_type == "direct":
        return independent_outputs(current_layer, source_names, num_channels, output_filter_size, padding, activation)
    elif output_type == "difference":
        cropped_input = crop(input, current_layer.get_shape().as_list(), match_feature_dim=False)
        return difference_output(cropped_input, current_layer, source_names, num_channels, output_filter_size, padding, activation, training)
    else:
        raise NotImplementedError("Unknown output type")

def independent_outputs(featuremap, source_names, num_channels, filter_width, padding, activation):
    outputs = dict()
    for name in source_names:
        outputs[name] = tf.keras.layers.Conv1D(num_channels, filter_width, activation=activation, padding=padding)(featuremap)
    return outputs


In [31]:
num_frames = 16384
num_channels = 1
num_layers = 12
num_initial_filters = 24
filter_size = 15
merge_filter_size = 5
input_filter_size = 15
output_filter_size = 1
padding = 'same'  
upsampling = 'linear'  # or 'learned'
output_type = 'direct'  # or 'difference'
source_names = ["accompaniment", "vocals"]
activation = 'tanh'
training = True

def build_model():
    # Input
    input_mix = Input(shape=(num_frames, num_channels), name="input")

    # Encoder
    enc_outputs = create_encoder(input_mix, num_layers, num_initial_filters, filter_size, input_filter_size, padding)

    # Decoder
    current_layer = create_decoder(enc_outputs, num_layers, num_initial_filters, filter_size, merge_filter_size, padding, upsampling)

    # Output Layer
    outputs = get_output_layer(current_layer, output_type, source_names, num_channels, output_filter_size, padding, activation, training)

    # Build Model
    model = Model(inputs=input_mix, outputs=outputs)
    return model

In [32]:

SAMPLE_RATE = 22050
SNIPPET_LENGTH = 16384  # Length of random snippets
AUGMENTATION = True    # Toggle data augmentation

def random_amplify(audio):
    factor = random.uniform(0.7, 1.3)  # Random amplification factor
    return audio * factor

def load_and_process_data(directory, min_mix=2, max_mix=5):
    X = []
    y = []
    vocal_dir = os.path.join(directory, '08Vox')
    other_dirs = [os.path.join(directory, folder) for folder in os.listdir(directory) if folder != '08Vox' and not folder.startswith('.')]
    
    for vocal_file in os.listdir(vocal_dir):
        if not vocal_file.lower().endswith(('.wav', '.mp3', '.flac')):
            continue
        
        vocal_path = os.path.join(vocal_dir, vocal_file)
        vocal_signal, _ = librosa.load(vocal_path, sr=SAMPLE_RATE)

        # Skip if the length is shorter than the snippet length
        if len(vocal_signal) < SNIPPET_LENGTH:
            continue

        # Normalize the vocal signal
        vocal_signal = normalize_audio(vocal_signal)

        # Randomly select a number of mixes
        num_mixes = random.randint(min_mix, max_mix)

        mixed_signal = vocal_signal.copy()  # Create a copy of the vocal signal to be mixed

        # Randomly select other samples to mix with the vocal
        for _ in range(num_mixes):
            other_dir = random.choice(other_dirs)
            other_file = random.choice([f for f in os.listdir(other_dir) if f.lower().endswith(('.wav', '.mp3', '.flac'))])
            other_path = os.path.join(other_dir, other_file)
            other_signal, _ = librosa.load(other_path, sr=SAMPLE_RATE)

            # Skip if the length is shorter than the snippet length
            if len(other_signal) < SNIPPET_LENGTH:
                continue

            other_signal = normalize_audio(other_signal)
            other_signal = pad_or_crop(other_signal, target_length=len(mixed_signal))
            mixed_signal += other_signal

        # Divide into segments of 16384 samples
        for i in range(0, len(vocal_signal), SNIPPET_LENGTH):
            vocal_segment = pad_or_crop(vocal_signal[i:i + SNIPPET_LENGTH], SNIPPET_LENGTH)
            mixed_segment = pad_or_crop(mixed_signal[i:i + SNIPPET_LENGTH], SNIPPET_LENGTH)

            X.append(mixed_segment)
            y.append(vocal_segment)

    return np.array(X), np.array(y)

def pad_or_crop(audio, target_length):
    length = len(audio)
    if length < target_length:
        padding = target_length - length
        audio = np.pad(audio, (0, padding), 'constant')
    elif length > target_length:
        audio = audio[:target_length]
    return audio

def normalize_audio(audio):
    return 2 * (audio - np.min(audio)) / (np.max(audio) - np.min(audio)) - 1


In [33]:
# Load and process the data for training and testing
X_train, y_train = load_and_process_data(trainDir)
X_test, y_test = load_and_process_data(testDir)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [34]:
# Importing necessary libraries to load the model
from tensorflow.keras.models import load_model

# Specifying the path to the trained model
model_path = "/Users/rei/Documents/Machine_Learning/MODELS/Unet/Shaking Through Sound Seperation/Shaking_Through_Model_01.keras"

Unet_Model = load_model(model_path, compile=False)

# Compile the model again with your desired optimizer
Unet_Model.compile(optimizer='adam', loss='mse')

model = build_model()  # Function that builds your model architecture
model.load_weights(model_path)
